In [70]:
import numpy as np
import pandas as pd
import nibabel as nib

In [71]:
# Display system information and library versions
import sys
print(f'python version: {sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'nibabel version: {nib.__version__}')
# Code was run on Windows 11 (MSI GS66 12UE: i7-12700H, 2.7 GHz, 16 GB)

python version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.16.5
pandas version: 0.25.1
nibabel version: 3.1.1


__Function Definitions__

In [72]:
def gaussian_distribution(mean, stdev, N=991):
    """
    Generates a Gaussian distribution function based on the inputed mean and 
    standard deviation of size N. 

    Inputs
    ------
    mean : float
        Mean of segment (i.e., from Mendez paper)
    
    std : float
        Standard deviation of segment (from Mendez paper)

    N : int 
        Number of points to make a Gaussian distribution for
    """
    X = np.arange(N)
    exp = np.exp((-1/2) * ((X-mean)/stdev)**2)
    denom = 1 / (stdev*np.sqrt(2*np.pi))
    
    return denom*exp

### Converting Anatomical Measurements to PAM50 Layers
Anatomical measurements used to estimate spinal levels are taken from [Mendez et al. (2021) - Segment-Specific Orientation of the Dorsal and Ventral Roots for Precise Therapeutic Targeting of Human Spinal Cord](https://www.sciencedirect.com/science/article/abs/pii/S0025619620310570)

Methodology: 
- Rostral and caudal extent of the spinal cord from the PAM50 was identified by a neurosurgeon to give a top and bottom layer of the template corresponding to C2 and L5. 
- The length of the spinal cord identified in Supplemental Table 2 from the Mendez paper was computed using the "Segment length at dorsal column entry in cm".
- The total length in cm was compared to the length in layers from the PAM50 to obtain a ratio, which was then used to multiply each length for spinal cord segment legnth and DREZ.


In [73]:
mendez_lengths = pd.read_excel('Mendez_lengths.xlsx')
display(mendez_lengths.head())
# TODO add level number correspondance
# For clarity, I separated the lengths and drez into separate dataframes
df_length = mendez_lengths[['Segment', 'length_mean_mm', 'length_stdev_mm']].reset_index()
df_length['Segment_id'] = df_length.index +2
df_drez = mendez_lengths[['Segment', 'drez_mean_mm', 'drez_stdev_mm']].reset_index()
df_length['Segment_id'] = df_drez.index +2

,Segment,length_mean_mm,length_stdev_mm,drez_mean_mm,drez_stdev_mm
0,C2,12.6,0.9,11.3,1.5
1,C3,13.3,0.7,9.4,1.9
2,C4,13.9,2.0,11.6,2.2
3,C5,15.0,2.2,12.4,2.0
4,C6,13.6,1.6,11.5,1.0


In [139]:
# Converting from mm to pixel values
PAM50_top_layer = 22  # as identified by a neurosurgeon (Superior-Inferior direction)
PAM50_bot_layer = 886 # as identified by a neurosurgeon (Superior-Inferior direction)
spine_legnth_mendez = mendez_lengths.length_mean_mm.sum()
spine_length_PAM50 = PAM50_bot_layer - PAM50_top_layer
print(spine_legnth_mendez, spine_length_PAM50*0.5)
ratio =  spine_length_PAM50 / spine_legnth_mendez
print(ratio)

412.70000000000005 432.0
2.093530409498425


In [75]:
# Get the cumulative sum of mean spinal segment lengths (gives the caudal side of each segment) 
df_length['cumulative_length_mm'] = np.cumsum(df_length.length_mean_mm)
# Get the midpoint of each spinal segment by subtracting 1/2 of the length to the midpoint
df_length['segment_midpoints_mm'] = df_length.cumulative_length_mm - df_length.length_mean_mm /2
df_length['segment_midpoints_pixel'] = df_length.segment_midpoints_mm * ratio + PAM50_top_layer 

# The midpoints are the same for drez
df_drez['segment_midpoints_pixel'] = df_length.segment_midpoints_pixel

# Convert the lengths and standard deviations into pixel values
df_length['length_mean_pixel'] = df_length.length_mean_mm * ratio
df_length['length_stdev_pixel'] = df_length.length_stdev_mm * ratio
df_drez['drez_mean_pixel'] = df_drez.drez_mean_mm * ratio
df_drez['drez_stdev_pixel'] = df_drez.drez_stdev_mm * ratio

0      35.189242
1      62.300460
2      90.772474
3     121.023988
4     150.961473
5     177.444633
6     202.776351
7     229.468864
8     260.243761
9     294.263630
10    334.982796
11    381.982554
12    431.075842
13    482.681367
14    535.019627
15    586.729828
16    638.230676
17    688.161376
18    730.869397
19    768.448268
20    801.421371
21    827.799855
22    852.608190
23    875.846378
Name: segment_midpoints_pixel, dtype: float64


In [76]:
# Get start and stop layers found by taking the midpoint of each segment and adding the 1/2 length and converting to int
df_length['layer_start'] = np.int32(df_length.segment_midpoints_pixel) - np.int32(df_length.length_mean_pixel / 2)
df_length['layer_stop'] = np.int32(df_length.segment_midpoints_pixel) + np.int32(df_length.length_mean_pixel / 2)

df_drez['layer_start'] = np.int32(df_drez.segment_midpoints_pixel) - np.int32(df_drez.drez_mean_pixel / 2)
df_drez['layer_stop'] = np.int32(df_drez.segment_midpoints_pixel) + np.int32(df_drez.drez_mean_pixel / 2)

### Creating new NIFTI files

In [77]:
df_length

,index,Segment,length_mean_mm,length_stdev_mm,Segment_id,cumulative_length_mm,segment_midpoints_mm,segment_midpoints_pixel,length_mean_pixel,length_stdev_pixel,layer_start,layer_stop
0,0,C2,12.6,0.9,2,12.6,6.30,35.189242,26.378483,1.884177,22,48
1,1,C3,13.3,0.7,3,25.9,19.25,62.300460,27.843954,1.465471,49,75
2,2,C4,13.9,2.0,4,39.8,32.85,90.772474,29.100073,4.187061,76,104
3,3,C5,15.0,2.2,5,54.8,47.30,121.023988,31.402956,4.605767,106,136
4,4,C6,13.6,1.6,6,68.4,61.60,150.961473,28.472014,3.349649,136,164
5,5,C7,11.7,1.7,7,80.1,74.25,177.444633,24.494306,3.559002,165,189
6,6,C8,12.5,1.6,8,92.6,86.35,202.776351,26.169130,3.349649,189,215
7,7,T1,13.0,2.8,9,105.6,99.10,229.468864,27.215895,5.861885,216,242
8,8,T2,16.4,5.0,10,122.0,113.80,260.243761,34.333899,10.467652,243,277
9,9,T3,16.1,3.0,11,138.1,130.05,294.263630,33.705840,6.280591,278,310


In [161]:
p50_nib_spinal_levels = nib.load('c:/Users/sb199/spinalcordtoolbox/data/PAM50/template/PAM50_label_spinal_levels.nii.gz')
spinal_z = np.where(p50_nib_spinal_levels.get_fdata()>0)[2]
spinal_z.sort()
display(np.array((990 - df_length['segment_midpoints_pixel']).to_list()).astype(int), np.flip(spinal_z))

array([954, 927, 899, 868, 839, 812, 787, 760, 729, 695, 655, 608, 558,
       507, 454, 403, 351, 301, 259, 221, 188, 162, 137, 114])

array([945, 927, 891, 857, 826, 794, 757, 713, 668, 622, 572, 522, 471,
       417, 365, 310, 251, 194, 169], dtype=int64)

In [78]:
# load the PAM50 mask
p50_nib = nib.load('c:/Users/sb199/spinalcordtoolbox/data/PAM50/template/PAM50_cord.nii.gz')
p50_mask = p50_nib.get_fdata()

In [137]:
# Spinal segment estimates
N = 991

for x, row in df_length.iterrows():
    # Copy mask and flip
    p50 = np.flip(np.copy(p50_mask))

    pdf_top = gaussian_distribution(row.layer_start, row.length_stdev_pixel)
    pdf_bot = gaussian_distribution(row.layer_stop, row.length_stdev_pixel)

    # Indices of probability density function > 1E-6
    ind_top = np.where(pdf_top > 1E-6)[0]
    ind_top = ind_top[ind_top < row.layer_start] # Only above estimated layer
    ind_bot = np.where(pdf_bot > 1E-6)[0]
    ind_bot = ind_bot[ind_bot > row.layer_stop] # Only below estiamted layer

    # Create a merged pdf with p=1 at estimated spinal cord levels
    pdf = np.zeros_like(pdf_top)
    pdf[row.layer_start:row.layer_stop+1] = np.max([pdf_top, pdf_bot])
    pdf[ind_top] = pdf_top[ind_top]
    pdf[ind_bot] = pdf_bot[ind_bot]

    # Set pdf < 1E-6 to 0
    ind = np.where(pdf > 1E-6)[0]
    zero_indices = np.arange(N)[~np.isin(np.arange(N), ind)]
    p50[:,:, zero_indices] = 0

    for i in ind:
        p50[:, :, i][p50[:, :, i]>0] = pdf[i]

    # Put back into the original orientation
    p50 = np.flip(p50)

    # Save as nifti
    if row.Segment_id < 10:
        row.Segment_id = '0'+ str(row.Segment_id)
    p50_img = nib.Nifti1Image(p50, header=p50_nib.header, affine=p50_nib.affine)
    nib.save(p50_img, f'Outputs/spinal_level_{row.Segment_id}.nii.gz')  # TO CHANGE path output


In [ ]:
# DREZ estimates

for x, row in df_drez.iterrows():
    # Copy mask and flip
    p50 = np.flip(np.copy(p50_mask))

    pdf_top = gaussian_distribution(row.layer_start, row.drez_stdev_pixel)
    pdf_bot = gaussian_distribution(row.layer_stop, row.drez_stdev_pixel)

    # Indices of probability density function > 1E-6
    ind_top = np.where(pdf_top > 1E-6)[0]
    int_top = ind_top[ind_top < row.layer_start] # Only above estimated layer
    ind_bot = np.where(pdf_bot > 1E-6)[0]
    ind_bot = ind_bot[ind_bot > row.layer_stop] # Only below estiamted layer

    # Create a merged pdf with p=1 at estimated spinal cord levels
    pdf = np.zeros_like(pdf_top)
    pdf[row.layer_start:row.layer_stop+1] = np.max([pdf_top, pdf_bot])
    pdf[ind_top] = pdf_top[ind_top]
    pdf[ind_bot] = pdf_bot[ind_bot]

    # Set pdf < 1E-6 to 0
    ind = np.where(pdf > 1E-6)[0]
    zero_indices = np.arange(N)[~np.isin(np.arange(N), ind)]
    p50[:,:, zero_indices] = 0

    for i in ind:
        p50[:, :, i][p50[:, :, i]>0] = pdf[i]

    # Put back into the original orientation
    p50 = np.flip(p50)

    # Save as nifti
    p50_img = nib.Nifti1Image(p50, affine=np.eye(4))
    nib.save(p50_img, f'Outputs/DREZ/Statistical_anatomical_{row.Segment}.nii.gz')



FileNotFoundError: [Errno 2] No such file or directory: 'Outputs/DREZ/Statistical_anatomical_C2.nii.gz'